In [1]:
NB_SIM = 3

In [2]:
import sys

sys.path.append('..')

In [3]:
def rec_dict_print(a_dict, level=1):
    keys = a_dict.keys()
    keys.sort()
    for k in keys:
        if type(a_dict[k]) is dict:
            print "".join(["-" for i in range(level)]) + k
            rec_dict_print(a_dict[k], level + 2)
        else:
            print "".join(["-" for i in range(level)]) + k

In [4]:
%matplotlib inline
import dill

loaded_data = None

with open('precomputed_sims/data%i.pkl' % (NB_SIM), 'rb') as f:
    loaded_data = dill.load(f)

In [5]:
rec_dict_print(loaded_data)

-N
-credit
---bc_ids
---bc_subsets_indexes
---copula
---default_times
-randomization
---distrib
-----loc
-----scale
---maturity
---zetas
-underlyings


In [6]:
N = loaded_data["N"]

In [7]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1 / 360.
delta = 5 * step

maturity = loaded_data["randomization"]["maturity"]

print "Maturity = %s years" % maturity

Maturity = 5.0 years


# Discount

In [8]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [9]:
udlyings = loaded_data["underlyings"]

print "Maximum number of paths: %i" % len(udlyings)

gbm0 = udlyings[0]

kappa = gbm0.drifts[0][0]
sigma = gbm0.vols[0][0]
print "kappa = %s, sigma = %s" % (kappa, sigma)

time_grid = gbm0.time

Maximum number of paths: 20000
kappa = 0.12, sigma = 0.2


# Derivative

In [10]:
derivatives_nb = 1

In [11]:
from finance.products.european.swap import (
    SwapContract,
)

swap_delta = 0.25

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(gbm0, discount, swap_dates)

price_0 = swap.price(0., incl_next_coupon=True)

print swap
print "\nPrice swap at t=0 without 1st coupon = %s" % price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 134.306, paying at {0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75, 5.00}

Price swap at t=0 without 1st coupon = 0.0


In [12]:
import numpy as np

p_fixed = 1.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

print "Notional on the swap: %s" % notional

Notional on the swap: 0.0015687485053


# Indexes stuffs

In [13]:
copula = loaded_data["credit"]["copula"]

c_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

obligors_nb = len(copula.subsets[c_subsets_indexes[-1]][0])
print "Obligor numbers: %s" % obligors_nb

c_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
c_positions = [0.69, -0.46, -0.44, -0.36, 0.34, 0.23, 0.09, -0.05, -0.04]

print "Counterparties id: %s (nb = %s)" % (c_ids, len(c_ids))

Obligor numbers: 125
Counterparties id: [17, 9, 29, 26, 50, 4, 5, 13, 64] (nb = 9)


In [14]:
positions = np.zeros(obligors_nb)
for idx, ps in zip(c_ids, c_positions):
    positions[idx] = ps

positions = positions / -positions[13]
positions = np.array(positions).reshape(positions.size, 1)

print positions.flatten()

[  0.    0.    0.    0.    4.6   1.8   0.    0.    0.   -9.2   0.    0.
   0.   -1.    0.    0.    0.   13.8   0.    0.    0.    0.    0.    0.
   0.    0.   -7.2   0.    0.   -8.8   0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    6.8   0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.   -0.8   0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0. ]


## Default times

In [15]:
default_times_mat = loaded_data["credit"]["default_times"]

In [16]:
import pandas as pd

def get_default_times_sim(c_ids, N, default_times_mat, copula, subsets_indexes):
    matrix_def = np.zeros((len(c_ids), N))
    c = np.array(c_ids)
    
    for j, c_id in enumerate(c_ids):
        cp_subsets_indexes = copula.get_indexes_including(c_id)
        c_df_times_indexes = [ii for ii, ind in enumerate(subsets_indexes) if ind in cp_subsets_indexes]
        
        c_df_times_mat = default_times_mat[:, c_df_times_indexes]
        c_df_times = c_df_times_mat.min(axis=1)
        
        matrix_def[j] = c_df_times
        
    matrix_def[matrix_def==1000.] = np.nan
        
    df_default_times = pd.DataFrame(matrix_def, index=c)
    return df_default_times

In [17]:
default_times = get_default_times_sim(c_ids, N, default_times_mat, copula, c_subsets_indexes)
default_times

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
17,4.697222,NaN,NaN,NaN,NaN,NaN,3.352778,NaN,3.155556,NaN,...,NaN,NaN,NaN,0.722222,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,0.680556,NaN,NaN,NaN,NaN,3.400000,4.472222,NaN,3.155556,NaN,...,NaN,NaN,NaN,0.722222,NaN,2.886111,NaN,NaN,NaN,NaN
26,NaN,1.847222,NaN,NaN,NaN,NaN,NaN,2.375000,1.100000,2.091667,...,NaN,NaN,NaN,0.722222,NaN,NaN,NaN,1.319444,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.658333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.938889,NaN,NaN,NaN,NaN,...,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.913889
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.155556,NaN,...,NaN,NaN,NaN,0.722222,NaN,NaN,NaN,NaN,2.358333,NaN


# Portfolio P&L

Code for:

\begin{equation*}
% \sum_{t < \tau_i^{\delta} \leq t+1} 
\left( 
\beta_{\tau_i^{\delta}} \left( MtM_{\tau_i^\delta}^i + \Delta^i_{\tau_i^\delta} \right)
-\beta_{\tau_i} \left( {\rm VM}_{\tau_i}^i + {\rm IM}_{\tau_i}^i \right)
\right)^+ \, \forall i
\end{equation*}

In [18]:
from utils import time_offseter

In [19]:
alpha = 0.70

In [20]:
times_cva = np.arange(0, maturity, 0.5)

shifted_times_cva = times_cva + 1.
shifted_times_cva[-1] = maturity

default_times.index.values

array([17,  9, 29, 26, 50,  4,  5, 13, 64], dtype=int64)

In [21]:
from functools import partial

def fact_(swap_, discount_, kappa_, delta_, t_):
    time_grid_ = swap_.underlying.time
    t_delta = time_offseter(t_ + delta_, time_grid_, True)
    
    coupon_dates_ = swap_.pillars
    l_t_delta_ = np.searchsorted(coupon_dates_, t_delta, side='left')

    beta_T_l_ = map(discount_, coupon_dates_[l_t_delta_ + 1 :])    
    h_l_ = swap_.delta_time[l_t_delta_ : ]
    
    T_l_m1_ = kappa_ * coupon_dates_[l_t_delta_ : -1]
    exp_factor_ = map(np.exp, T_l_m1_)
    
    tmp_ = np.multiply(exp_factor_, h_l_)    
    res = np.dot(beta_T_l_, tmp_)
    
    return res

fact = partial(fact_, swap, discount, kappa, delta)

In [22]:
def compute_1y_ahead_loss_(fact_f_, kappa_, delta_, notional_, positions_, times_cva_, shifted_times_cva_, default_times_, udls_):
    ccp_loss_1y_ = dict()
    
    N_ = len(udls_)
    
    c_ids_ = default_times_.index.values
    
    for t0_, t1_ in zip(times_cva_, shifted_times_cva_):
        key = "[%.2f, %.2f]" % (t0_, t1_)
        print key
        ccp_loss_1y_[key] = pd.DataFrame(0., index=range(N_), columns=c_ids_)
        
        for j_ in xrange(N_):
            udl_ = udls_[j_]
            time_grid_ = udl_.time
            tau_j_ = default_times_.loc[c_ids_, j_]
            
            c_defaulted_ = tau_j_[(tau_j_ > t0_) & (tau_j_ <= t1_)]
            for (c_idx_, tau_) in c_defaulted_.iteritems():
                tau_ = time_offseter(tau_, time_grid_, True)
                tau_delta_ = time_offseter(tau_ + delta_, time_grid_, True)
                
                hat_s_tau_ = udl_(tau_)[0][0] * np.exp(kappa_ * tau_)
                hat_s_tau_delta_ = udl_(tau_delta_)[0][0] * np.exp(kappa_ * tau_delta_)
            
                loss_c_ = notional_ * fact_f_(tau_) * positions_[c_idx_][0] * (hat_s_tau_ - hat_s_tau_delta_)
                
                loss_c_ = np.maximum(loss_c_, 0.)
            
                ccp_loss_1y_[key].loc[j_, c_idx_] = loss_c_

    return ccp_loss_1y_
            
compute_1y_ahead_loss = partial(compute_1y_ahead_loss_, fact, kappa, delta, notional, positions)

In [23]:
import os

loss_path = './res/sim%i/loss_1y_ahead/' % (NB_SIM)

if not os.path.isdir(loss_path):
    os.makedirs(loss_path)
    one_y_ahead_loss = compute_1y_ahead_loss(times_cva, shifted_times_cva, default_times, udlyings)
    
    for k, dataframe_ in one_y_ahead_loss.iteritems():
        path = os.path.join(loss_path, 'loss_%s.csv' % k)
        dataframe_.to_csv(path)

[0.00, 1.00]
[0.50, 1.50]
[1.00, 2.00]
[1.50, 2.50]
[2.00, 3.00]
[2.50, 3.50]
[3.00, 4.00]
[3.50, 4.50]
[4.00, 5.00]
[4.50, 5.00]


In [24]:
from scipy.stats import norm

def B_(fact_f_, vol_, delta_, a_, omega_, t_):
    if a_ <= 0.5: 
        a_ = 1. - a_
    
    perc_ = a_ if omega_ <= 0. else (1. - a_)
    q_ = norm.ppf(perc_)
    
    var_ = vol_**2 * delta_
    exp_factor_ = np.exp(-0.5 * var_ + np.sqrt(var_) * q_)
    
    tmp_res_ = 1. - exp_factor_
    mult = np.sign(omega_)
    
    return mult * tmp_res_ * fact_f_(t_)

B = partial(B_, fact, sigma, delta, alpha)

def IM_(B_f, discount_f, kappa_, nom_, omega_, udl_, t_):
    nom_i_ = nom_ * np.fabs(omega_)
    b_t_ = B_f(omega_, t_)

    hat_S_t_ = np.exp(kappa_ * t_) * udl_(t_)[0][0]
    
    return nom_i_ * b_t_ * hat_S_t_ / discount_f(t_)

IM = partial(IM_, B, discount, kappa)

In [25]:
def A_(fact_f_, vol_, delta_, a_, omega_, t_):
    if a_ <= 0.5: 
        a_ = 1. - a_
    
    var_ = vol_**2 * delta_
    
    perc_ = a_ if omega_ <= 0. else (1. - a_)
    q_ = norm.ppf(perc_)
    sqrt_var_ = np.sqrt(var_)
    var_factor_ = np.exp(sqrt_var_ * q_)
    
    sgn_omega_ = np.sign(omega_)    
    es_gauss_ = - norm.pdf(norm.ppf(a_)) / (1. - a_) * sgn_omega_
    es_factor_ = np.exp(sqrt_var_ * es_gauss_)
    
    diff_term_ = (var_factor_ - es_factor_) * sgn_omega_
    
    comm_factor_ = (1. - a_) * fact_f_(t_) * np.exp(-0.5 * var_)
    
    res_ = comm_factor_ * diff_term_
    return res_

A = partial(A_, fact, sigma, delta, alpha)

In [26]:
def E_(fact_f_, kappa_, vol_, delta_, omega_, udl_, tau_, t_):
    t_ = time_offseter(t_, udl_.time, True)
    tau_ = time_offseter(tau_, udl_.time, True)
    tau_delta_ = time_offseter(tau_ + delta_, udl_.time, True)
    
    assert tau_ < t_ and t_ < tau_delta_, "t should belong to [tau, tau^delta]"
    
    hat_S_t_ = np.exp(kappa_ * t_) * udl_(t_)[0][0]
    hat_S_tau_ = np.exp(kappa_ * tau_) * udl_(tau_)[0][0]
    
    std_dev_ = vol_ * np.sqrt(tau_delta_ - t_)
    y_ = np.log(hat_S_t_ / hat_S_tau_) / std_dev_
    y_minus_ = y_ - 0.5 * std_dev_
    y_plus_ = y_ + 0.5 * std_dev_
    
    sgn_omega_ = np.sign(omega_)
    s_t_phi_ = hat_S_t_ * norm.cdf(- y_plus_ * sgn_omega_)
    s_tau_phi_ = hat_S_tau_ * norm.cdf(- y_minus_ * sgn_omega_)
    
    res_ = fact_f_(tau_) * (s_tau_phi_ - s_t_phi_) * sgn_omega_
    
    return res_

E = partial(E_, fact, kappa, sigma, delta)

In [27]:
#from scipy.integrate import quad
from scipy.integrate import simps

def CVA_(A_f_, E_f_, copula_, discount_f, kappa_, delta_, nom_, swap_, c_id_, c_pos_, c_tau_, t_):
    udl_ = swap_.underlying
    t_ = time_offseter(t_, udl_.time, True)
    hat_S_t_ = np.exp(kappa_ * t_) * udl_(t_)[0][0]
    
    if np.isnan(c_tau_):
        return 0.
    
    c_tau_ = time_offseter(c_tau_, udl_.time, True)
    inv_surv_proba_t_ = 1. / copula_.tot_survival_proba(t_, c_id_)
    nom_i_ = np.fabs(c_pos_) * nom_
    
    if t_ < c_tau_:
        def integrand(s_):
            gamma_s_ = copula_.tot_gamma(s_, c_id_)
            surv_proba_s_ = copula_.tot_survival_proba(s_, c_id_)
            return A_f_(c_pos_, s_) * gamma_s_ * surv_proba_s_
            
        # Can't call directly
        # integral_ = quad(integrand, t_, mat_)[0]
        # because the integrand jumps at the coupon times
        # and the intensity calibration times
        sub_indexes_ = copula_.get_indexes_including(c_id_)
        int_pillars_ = set(copula_.pillars[sub_indexes_].flatten())
        swap_pillars_ = set(swap_.pillars)
            
        cut_times_ = list(int_pillars_ | swap_pillars_ | set([t_]))
        cut_times_.sort()
            
        cut_times_ = np.array(cut_times_)
        cut_times_ = cut_times_[t_ <= cut_times_]

        integral_ = 0.
        for t_i_, t_ip1_ in zip(cut_times_[:-1], cut_times_[1:]):
            #integral_ += quad(integrand, t_i_, t_ip1_)[0]
            x__ = np.linspace(t_i_, t_ip1_, 10)
            y__ = [integrand(x_i_) for x_i_ in x__]
            tmp_int__ = simps(y__, x__)
            integral_ += tmp_int__

        return nom_i_ * hat_S_t_ * inv_surv_proba_t_ * integral_ / discount_f(t_)
        
    c_tau_delta_ = time_offseter(c_tau_ + delta_, udl_.time, True)
    if c_tau_ < t and t < c_tau_delta_:
        return nom_i_ * E_f_(c_pos_, udl_, tau_i_, t_) / discount_f(t_)
    
CVA = partial(CVA_, A, E, copula, discount, kappa, delta, notional)

In [28]:
%timeit CVA(swap, 17, 4.6, 3.2, 0.)

1 loop, best of 3: 220 ms per loop


In [29]:
def compute_cva(swap_, times_cva_, default_times_, udls_, all_positions_):
    ccp_cva = dict()
    
    N_ = len(udls_)
    
    c_ids_ = default_times_.index.values
    
    for t_ in times_cva_:
        key = "%.2f" % t_
        print key
        ccp_cva[key] = pd.DataFrame(0., index=range(N_), columns=c_ids_)
        
        for j_ in xrange(N_):
            udl_ = udls_[j_]
            swap_.underlying = udl_
            
            for c_id_ in c_ids_:
                c_pos_ = all_positions_[c_id_][0]
                c_tau_ = default_times_.loc[c_id_, j_]
                ccp_cva[key].loc[j_, c_id_] = CVA(swap_, c_id_, c_pos_, c_tau_, t_)
                
    return ccp_cva

In [30]:
cva_path = './res/sim%i/cva_ccp/' % (NB_SIM)

if not os.path.isdir(cva_path):
    os.makedirs(cva_path)
    cva_ccp = compute_cva(swap, times_cva, default_times, udlyings, positions)
    
    for k, dataframe_ in cva_ccp.iteritems():
        path = os.path.join(cva_path, 'cva_%s.csv' % k)
        dataframe_.to_csv(path)

0.00
0.50
1.00
1.50
2.00
2.50
3.00
3.50
4.00
4.50
